<a href="https://colab.research.google.com/github/brauner68/gm_proj/blob/main/trainer_debug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Drive & Repo

In [ ]:
import os
from google.colab import drive

# 1. Mount Drive (Your Data)
drive.mount('/content/drive')

# 2. Sync Code from GitHub
REPO_URL = "https://github.com/brauner68/gm_proj.git"
REPO_NAME = "gm_proj"

if not os.path.exists(f"/content/{REPO_NAME}"):
    print(f"⬇️ Cloning {REPO_NAME}...")
    !git clone {REPO_URL}
else:
    print(f"🔄 Repo exists. Pulling latest changes...")
    %cd /content/{REPO_NAME}
    !git pull

# 3. Enter Project Folder
%cd /content/{REPO_NAME}

# 4. Install Dependencies (The Magic Step)
# This reads your requirements.txt and installs everything at once
print("📦 Installing libraries from requirements.txt...")
!pip install -q -r requirements.txt

print("\n✅ SETUP COMPLETE! You are ready to work.")

# Unzip Data

In [ ]:
# Create a folder for the unzipped data
!mkdir -p /content/nsynth_data

# Unzip the file (This might take a minute)
# The -C flag tells it where to put the files
print("⏳ Unzipping data...")
!tar -xzf "/content/drive/Shareddrives/gm_proj/nsynth-valid.jsonwav.tar.gz" -C /content/nsynth_data
print("✅ Done!")

# Train

In [ ]:
from src.trainer import DiffusionTrainer

# 1. Configuration
config = {
    'data_path': '/content/nsynth_data/nsynth-valid',       # Where you unzipped the data
    'max_samples': 1000,                                    # How many samples to load
    'output_dir': '/content/gm_proj/results/run_01',        # Where to save models/images
    'selected_families': ['guitar', 'mallet', 'brass'],     # The instruments you want
    'epochs': 15,                                            # How long to train
    'batch_size': 16,                                       # Adjust based on GPU memory
    'lr': 1e-4,                                             # Learning Rate
    'save_interval': 5,                                     # Save every 5 epochs
    'cfg_prob': 0.1                                         # 10% Dropout for CFG
}

# 2. Run
print("🚀 Starting Training...")
trainer = DiffusionTrainer(config)
trainer.train()

# Test Audio

In [ ]:
from src.vocoder import Vocoder
from src.dataset import NSynthDataset
import torch
import IPython.display as ipd

# 1. Initialize
vocoder = Vocoder(device='cuda' if torch.cuda.is_available() else 'cpu')

# Load Spectogram
valid_path = '/content/nsynth_data/nsynth-valid'
dataset = NSynthDataset(
    data_path=valid_path,
    max_samples=None,
    selected_families=['mallet']
)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)
images, labels = next(iter(dataloader))
spec = images[0]

# 3. Convert to Audio
audio = vocoder.decode(spec)

# 4. Save and Play
save_path = "/content/test_tone.wav"
vocoder.save_audio(audio, save_path)

print("Playing Audio:")
ipd.Audio(save_path)